In [0]:
from google.colab import files
uploaded = files.upload()

Saving EEVR_D-Evrak_iletisim_detay.xlsx to EEVR_D-Evrak_iletisim_detay.xlsx


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_excel('EEVR_D-Evrak_iletisim_detay.xlsx')

In [0]:
df = df.drop(columns={'Birim Adı','Evrak Türü','Doküman No','Kapsam','Geliş Şekli','Evrak Zaman','Evrak Sonuç Zaman','Hedef/Miat Tarihi','Evrak Sonuç Durum','Doküman Adres Pafta','Doküman Adres Mahalle','Doküman Adres Ada','Doküman Adres Parsel','Doküman Yer Adresi','Süre Durumu','İşlem Durumu','İşlem Yapılan Konu','İşlem Yapılan Alt Konu','İşlem Yapılan Süre','İşlem Yapılan Süre Birimi','Kurum Görüşü'})
df = df.dropna(axis=0)
df = df.drop_duplicates()
df = df.reset_index(drop=True)
df.shape


(13711, 2)

In [0]:
sayi = df['İşlem Süresi'].count() #for döngüsü için kaç tane değer olduğunu aldık
time = [df['İşlem Süresi'][i][-18:-16] for i in range(sayi)] #İşlem Süresi kolonundaki değerlerin sadece gün sayılarını aldık

df['İşlem Süresi'] = time

In [0]:

print(df['İşlem Süresi'].value_counts().head(15))

0     4548
1     1741
2     1696
3     1141
4      931
5      664
6      511
7      398
8      279
9      216
10     181
11     145
12     130
13      92
14      81
Name: İşlem Süresi, dtype: int64


In [0]:
#print(df['İşlem Süresi'].value_counts().head(15))

for i in range(len(df['İşlem Süresi'])):
  if int(df['İşlem Süresi'][i]) < 1:
    df['İşlem Süresi'][i] = '1'
    
  elif int(df['İşlem Süresi'][i]) < 4 and int(df['İşlem Süresi'][i]) >= 1:
    df['İşlem Süresi'][i] = '3'
    
  elif int(df['İşlem Süresi'][i]) <= 10 and int(df['İşlem Süresi'][i]) >= 4:
    df['İşlem Süresi'][i] = '5'
  
  else:
    df['İşlem Süresi'][i] = '10'

In [0]:
import os
newpath = '/content/file' 
if not os.path.exists(newpath):
    os.makedirs(newpath)

In [0]:
df = df.sample(frac=1)

train_df = df[:12000]
train_df = train_df.reset_index(drop=True)

test_df = df[12000:13000]
test_df = test_df.reset_index(drop=True)

dev_df = df[13000:]
dev_df = dev_df.reset_index(drop=True)

In [0]:
train_df_bert = pd.DataFrame({
    'id':range(len(train_df)),
    'label':train_df['İşlem Süresi'],
    'alpha':['a']*train_df['İşlem Süresi'].shape[0],
    'text': train_df['Doküman Metin'].replace(r'\n', ' ', regex=True)# train_df_bert diye yeni bir dataframe oluşturduk berte vermeden önce bu hale getirmemiz gerekiyormuş
})

In [0]:
dev_df_bert = pd.DataFrame({
    'id':range(len(dev_df)),
    'label':dev_df['İşlem Süresi'],
    'alpha':['a']*dev_df['İşlem Süresi'].shape[0],
    'text': dev_df['Doküman Metin'].replace(r'\n', ' ', regex=True)# train_df_bert diye yeni bir dataframe oluşturduk berte vermeden önce bu hale getirmemiz gerekiyormuş
})

In [0]:
test_df_bert = pd.DataFrame({
    'id':range(len(test_df)),
    'label':test_df['İşlem Süresi'],
    'alpha':['a']*test_df['İşlem Süresi'].shape[0],
    'text': test_df['Doküman Metin'].replace(r'\n', ' ', regex=True)# train_df_bert diye yeni bir dataframe oluşturduk berte vermeden önce bu hale getirmemiz gerekiyormuş
})

In [0]:
train_df_bert.head().shape

(5, 4)

In [0]:
train_df_bert.to_csv('/content/file/train.tsv', index=False, sep="\t")
test_df_bert.to_csv('/content/file/test.tsv', index=False, sep="\t")
dev_df_bert.to_csv('/content/file/dev.tsv', index=False, sep="\t")

In [0]:
!pip install flair

     |████████████████████████████████| 184kB 8.8MB/s 
     |████████████████████████████████| 655kB 57.6MB/s 
     |████████████████████████████████| 163kB 27.3MB/s 
     |████████████████████████████████| 1.0MB 24.2MB/s 
     |████████████████████████████████| 748.9MB 25kB/s 
     |████████████████████████████████| 778kB 50.4MB/s 
     |████████████████████████████████| 798kB 49.2MB/s 
     |████████████████████████████████| 1.0MB 57.8MB/s 
     |████████████████████████████████| 337kB 56.1MB/s 
  Created wheel for sqlitedict: filename=sqlitedict-1.6.0-cp36-none-any.whl size=14689 sha256=4d7ab31c853d70183a8c67c34d0e0191542ac79f13ab3cc1e947869fdf5b1b05
  Stored in directory: /root/.cache/pip/wheels/bd/57/d3/907c3ee02d35e66f674ad0106e61f06eeeb98f6ee66a6cc3fe
  Created wheel for regex: filename=regex-2019.8.19-cp36-cp36m-linux_x86_64.whl size=609221 sha256=44bd6cf9430e545c3b29246cf0fec69cdc4cc02e59d8708dda0317e815513a9b
  Stored in directory: /root/.cache/pip/wheels/90/04/07/b5010fb8167

In [0]:
from flair.data import Corpus
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings
from typing import List

# 1. get the corpus
from flair.data import Corpus
from flair.datasets import CSVClassificationCorpus

# this is the folder in which train, test and dev files reside
data_folder = '/content/file'

# column format indicating which columns hold the text and label(s)
column_name_map = {3: "text", 1: "label", 2: "id"}

# load corpus containing training, test and dev data and if CSV has a header, you can skip it
corpus: Corpus = CSVClassificationCorpus(data_folder,
                                         column_name_map,
                                         skip_header=True,
                                         delimiter='\t',    # tab-separated files
)  

  
# 2. what tag do we want to predict?
tag_type = 'ner'


# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary('label')
print(tag_dictionary.idx2item)

# 4. initialize embeddings
embedding_types: List[TokenEmbeddings] = [

    WordEmbeddings('glove'),

    # comment in this line to use character embeddings
    # CharacterEmbeddings(),

    # comment in these lines to use flair embeddings
    # FlairEmbeddings('news-forward'),
    # FlairEmbeddings('news-backward'),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

# 5. initialize sequence tagger
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)

# 6. initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

# 7. start training
trainer.train('resources/taggers/example-ner',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=10)

# 8. plot weight traces (optional)
from flair.visual.training_curves import Plotter
plotter = Plotter()
plotter.plot_weights('resources/taggers/example-ner/weights.txt')

2019-08-28 15:21:02,994 Reading data from /content/file
2019-08-28 15:21:02,995 Train: /content/file/train.tsv
2019-08-28 15:21:02,996 Dev: /content/file/dev.tsv
2019-08-28 15:21:02,997 Test: /content/file/test.tsv
[b'<unk>', b'O', b'', b'<START>', b'<STOP>']


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


2019-08-28 15:21:13,667 ----------------------------------------------------------------------------------------------------
2019-08-28 15:21:13,669 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('glove')
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=100, out_features=100, bias=True)
  (rnn): LSTM(100, 256, bidirectional=True)
  (linear): Linear(in_features=512, out_features=5, bias=True)
)"
2019-08-28 15:21:13,671 ----------------------------------------------------------------------------------------------------
2019-08-28 15:21:13,671 Corpus: "Corpus: 12000 train + 711 dev + 1000 test sentences"
2019-08-28 15:21:13,672 ----------------------------------------------------------------------------------------------------
2019-08-28 15:21:13,673 Parameters:
2019-08-28 15:21:13,674  - learning_rate: "0.1"
2019-08-28 15:21:13,675  - mini_batch_size: "32"
2019-08-28 

Process Process-323:
Process Process-319:


2019-08-28 15:32:25,987 ----------------------------------------------------------------------------------------------------
2019-08-28 15:32:25,998 Exiting from training early.
2019-08-28 15:32:26,003 Saving model ...


Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 261, in _bootstrap
    util._exit_function()
  File "/usr/lib/python3.6/multiprocessing/util.py", line 322, in _exit_function
    _run_finalizers()
  File "/usr/lib/python3.6/multiprocessing/util.py", line 262, in _run_finalizers
    finalizer()
  File "/usr/lib/python3.6/multiprocessing/util.py", line 186, in __call__
    res = self._callback(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 191, in _finalize_join
    thread.join()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 261, in _bootstrap
    util._exit_function()
  File "/usr/lib/python3.6/threading.py", line 1056, in join
    self._wait_for_tstate_lock()
  File "/usr/lib/python3.6/threading.py", line 1072, in _wait_for_tstate_lock
    elif lock.acquire(block, timeout):
KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing/util.py", line 

KeyboardInterrupt: ignored

In [0]:
a=corpus.obtain_statistics()
print(a)

{
    "TRAIN": {
        "dataset": "TRAIN",
        "total_number_of_documents": 12000,
        "number_of_documents_per_class": {
            "3": 4032,
            "1": 3964,
            "5": 2769,
            "10": 1235
        },
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 393015,
            "min": 1,
            "max": 491,
            "avg": 32.75125
        }
    },
    "TEST": {
        "dataset": "TEST",
        "total_number_of_documents": 1000,
        "number_of_documents_per_class": {
            "5": 245,
            "1": 340,
            "3": 319,
            "10": 96
        },
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 33827,
            "min": 1,
            "max": 441,
            "avg": 33.827
        }
    },
    "DEV": {
        "dataset": "DEV",
        "total_number_of_documents": 711,
        "number_of_documents_per_class": {
            "1": 244,
            "3": 

In [0]:
def Sentence(texts):
  tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
  tokenizer.fit_on_texts(texts)
  sequences = tokenizer.texts_to_sequences(texts)

  word_index = tokenizer.word_index
  print('Number of Unique Tokens',len(word_index))

In [0]:
for i in len(df['İşlem Süresi']):
  df[]

,Doküman Metin,İşlem Süresi


In [0]:
print(df['İşlem Süresi'][13658])
df['Doküman Metin'][13658]


10


'Merhaba, Kadıköyde yapılan yeni binaların çoğunun içerisinde acil çıkış kapısı var. Bu hırsızlara davet çıkarıyor ki duyuyoruz bu sebepten olan hırsızlıkları. Bu konuda bir düzenleme yapacak mısınız? Ev tutmaya çalışıyoruz ama bu sebepten ev bulamıyoruz.'

In [0]:
model = SequenceTagger.load('resources/taggers/example-ner/final-model.pt')

# create example sentence
sentence = Sentence('I love Berlin')

# predict tags and print
model.predict(sentence)

print(sentence.to_tagged_string())

2019-08-28 17:11:36,729 loading file resources/taggers/example-ner/final-model.pt


EOFError: ignored